In [63]:
using ModelingToolkit, Sophon, DomainSets, TaylorDiff
using Optimization, OptimizationOptimJL

@parameters t, x
@variables u(..)
Dt = Differential(t)
Dx = Differential(x)
Dxx = Differential(x)^2

r1 = 0.02
r2 = 1

eq = Dt(u(t, x)) - Dxx(r2*(u(t,x)^3 - u(t,x)) - r1 * Dxx(u(t,x))) ~ 0.0

bcs = [u(0, x) ~ cospi(x)-exp(-4*π*π*abs2(x)),
    u(t, -1.0) ~ u(t, 1.0),
    Dx(u(t, -1.0)) ~ Dx(u(t, 1.0))]

domains = [t ∈ Interval(0.0, 1.0),
    x ∈ Interval(-1.0, 1.0)]

@named pde_system = PDESystem(eq, bcs, domains, [t,x], [u(t,x)])

PDESystem
Equations: Equation[Differential(t)(u(t, x)) - Differential(x)(Differential(x)(u(t, x)^3 - 0.02Differential(x)(Differential(x)(u(t, x))) - u(t, x))) ~ 0.0]
Boundary Conditions: Equation[u(0, x) ~ cospi(x) - exp(-39.47841760435743abs2(x)), u(t, -1.0) ~ u(t, 1.0), Differential(x)(u(t, -1.0)) ~ Differential(x)(u(t, 1.0))]
Domain: Symbolics.VarDomainPairing[Symbolics.VarDomainPairing(t, 0.0..1.0), Symbolics.VarDomainPairing(x, -1.0..1.0)]
Dependent Variables: Num[u(t, x)]
Independent Variables: Num[t, x]
Parameters: SciMLBase.NullParameters()
Default Parameter ValuesDict{Any, Any}()

In [104]:
chain = FullyConnected(2, 1, tanh; hidden_dims=16, num_layers=5)
pinn = PINN(chain)
sampler = BetaRandomSampler(1000, (400, 100, 100); α=0.4)
strategy = NonAdaptiveTraining(1, (1000,10,10))
prob = Sophon.discretize(pde_system, pinn, sampler, strategy)

OptimizationProblem. In-place: true
u0: ComponentVector{Float64}(layer_1 = (weight = [0.7632691860198975 -1.5043280124664307; -0.40536874532699585 1.8406134843826294; … ; 1.5719157457351685 -0.08423525840044022; -1.3312100172042847 1.4611992835998535], bias = [0.0; 0.0; … ; 0.0; 0.0;;]), layer_2 = (weight = [0.006376511417329311 -0.37400591373443604 … -0.1611759215593338 0.2889919579029083; -0.6003273129463196 -0.09576312452554703 … 0.346310019493103 0.5229037404060364; … ; 0.33052653074264526 -0.47113728523254395 … -0.6552205085754395 -0.029657170176506042; 0.4268585741519928 -0.1359691023826599 … 0.1872396320104599 -0.63426673412323], bias = [0.0; 0.0; … ; 0.0; 0.0;;]), layer_3 = (weight = [-0.48864418268203735 0.42416200041770935 … -0.5922123193740845 -0.37207967042922974; 0.2782418429851532 -0.525311291217804 … -0.3537571132183075 -0.3609273433685303; … ; -0.45926806330680847 -0.1884203404188156 … 0.6809808015823364 -0.6483128070831299; 0.6179205775260925 -0.6968308687210083 … -0.0

In [105]:
i = 0
function callback(p, l)
    global i
    i%100==0 && println("Loss: $l")
    i += 1
    return false
end

res = Optimization.solve(prob, BFGS(); maxiters=1000, callback=callback)

In [ ]:
using CairoMakie, MakiePublication

phi = pinn.phi
ts, xs = [infimum(d.domain):0.01:supremum(d.domain) for d in pde_system.domain]
axis = (xlabel=L"t", ylabel=L"x", title="Prediction")
u_pred = [sum(pinn.phi([t, x], res.u)) for t in ts, x in xs]
heatmap(ts, xs, u_pred, axis=axis, colormap=:jet)

In [101]:
for α in [0.6, 0.8, 1.0] 
    i = 0
    sampler = remake(sampler; α=α)
    data = Sophon.sample(pde_system, sampler)
    prob = remake(prob; p=data, u0=res.u)
    res = Optimization.solve(prob, BFGS(); maxiters=1000, callback=callback)

    u_pred = [sum(pinn.phi([t, x], res.u)) for t in ts, x in xs]
    heatmap(ts, xs, u_pred, axis=axis, colormap=:jet)
    display(fig)
end

InterruptException: InterruptException: